# Batch Document Extraction with Llama Vision (V2)

Streamlined batch processing notebook using modular components.

**V2 Features:**
- **Sophisticated bank statement extraction** using multi-turn UnifiedBankExtractor
- Turn 0: Header detection (identifies actual column names)
- Turn 1: Adaptive extraction with structure-dynamic prompts
- Automatic strategy selection: BALANCE_DESCRIPTION, AMOUNT_DESCRIPTION, etc.
- Early model loading
- Configurable output directory
- Comprehensive analytics and visualizations
- Clean, modular code structure

**Bank Statement Processing Toggle:**
- `USE_SOPHISTICATED_BANK_EXTRACTION`: True (default) uses multi-turn extraction
- `ENABLE_BALANCE_CORRECTION`: Optional mathematical balance validation

## 1. Imports

In [1]:
#Cell 1
# Project root detection - allows running from subdirectories
import sys
import os
from pathlib import Path

os.environ['EVALUATION_METHOD'] = 'order_aware_f1'  # or 'f1', 'kieval', 'order_aware_f1', 'correlation'

def find_project_root(marker_files=('common/__init__.py', 'CLAUDE.md', 'prompts')):
    """Find project root by looking for marker files, searching up from cwd."""
    current = Path().absolute()
    for parent in [current] + list(current.parents):
        for marker in marker_files:
            if (parent / marker).exists():
                return parent
    return current  # Fallback to cwd if no marker found

# Detect project root (works from any subdirectory)
PROJECT_ROOT = find_project_root()
print(f"📂 Current directory: {Path().absolute()}")
print(f"🏠 Project root: {PROJECT_ROOT}")

# Ensure the project root is in the Python path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    print(f"✅ Added {PROJECT_ROOT} to sys.path")

# Enable autoreload for module changes
%load_ext autoreload
%autoreload 2

# Standard library imports
import warnings
from datetime import datetime

# Third-party imports
import numpy as np
import pandas as pd
from IPython.display import display, Image
from rich import print as rprint
from rich.console import Console

# Project-specific imports
from common.batch_analytics import BatchAnalytics
from common.batch_processor import BatchDocumentProcessor
from common.batch_reporting import BatchReporter
from common.batch_visualizations import BatchVisualizer
from common.evaluation_metrics import load_ground_truth
from common.extraction_parser import discover_images
from common.gpu_optimization import emergency_cleanup
from common.llama_model_loader_robust import load_llama_model_robust

# V2: Sophisticated bank statement processing
from common.bank_statement_adapter import BankStatementAdapter

print("✅ All imports loaded successfully")
print("✅ Llama batch processing modules imported successfully")
print("✅ V2: BankStatementAdapter imported for sophisticated bank extraction")
warnings.filterwarnings('ignore')

📂 Current directory: /home/jovyan/nfs_share/tod/LMM_POC/notebooks_v2
🏠 Project root: /home/jovyan/nfs_share/tod/LMM_POC
✅ Added /home/jovyan/nfs_share/tod/LMM_POC to sys.path


FileNotFoundError: Field definitions not found: config/field_definitions.yaml

## 2. Pre-emptive Memory Cleanup

**CRITICAL for V100**: Run this cell first to prevent OOM errors when switching between models.

In [ ]:
#Cell 2
# Initialize console for rich output
console = Console()# Pre-emptive V100 Memory Cleanup - Run FIRST to prevent OOM errors
rprint("[bold red]🧹 PRE-EMPTIVE V100 MEMORY CLEANUP[/bold red]")
rprint("[yellow]Clearing any existing model caches before loading...[/yellow]")
rprint("[cyan]💡 This prevents OOM errors when switching between models on V100[/cyan]")

# Emergency cleanup to ensure clean slate
emergency_cleanup(verbose=True)

rprint("[green]✅ Memory cleanup complete - ready for model loading[/green]")
rprint("[dim]📋 Next: Import modules and configure settings[/dim]")

## 3. Configuration

In [ ]:
#Cell 3
# Environment-specific base paths
ENVIRONMENT_BASES = {
    'sandbox': '/home/jovyan/nfs_share/tod',
    'efs': '/efs/shared/PoC_data'
}
base_data_path = ENVIRONMENT_BASES['sandbox']

CONFIG = {
    # Model settings
    # 'MODEL_PATH': "/efs/shared/PTM/Llama-3.2-11B-Vision-Instruct",
    'MODEL_PATH': "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision-Instruct",

    # Batch settings - Using base path for consistency
    'DATA_DIR': f'{base_data_path}/LMM_POC/evaluation_data/synthetic',
    'GROUND_TRUTH': f'{base_data_path}/LMM_POC/evaluation_data/synthetic/ground_truth_synthetic.csv',
    'OUTPUT_BASE': f'{base_data_path}/LMM_POC/output',
    'MAX_IMAGES': None,  # None for all, or set limit
    'DOCUMENT_TYPES': None,  # None for all, or ['invoice', 'receipt']
    'ENABLE_MATH_ENHANCEMENT': True,  # Enable mathematical correction for bank statements
    
    # Inference and evaluation mode
    'INFERENCE_ONLY': False,  # Default: True (inference-only mode)

    # Verbosity control
    'VERBOSE': True,
    'SHOW_PROMPTS': True,

    # L40S optimization settings
    'USE_QUANTIZATION': False,
    'DEVICE_MAP': 'auto',
    'MAX_NEW_TOKENS': 3000,  # Reduced for L40S (was 4000 for V100)
    'TORCH_DTYPE': 'bfloat16',
    'LOW_CPU_MEM_USAGE': True,
    
    # ============================================================================
    # V2: SOPHISTICATED BANK STATEMENT EXTRACTION
    # ============================================================================
    'USE_SOPHISTICATED_BANK_EXTRACTION': True,
    'ENABLE_BALANCE_CORRECTION': True,
    
    # ============================================================================
    # IMAGE PREPROCESSING SETTINGS
    # ============================================================================
    'ENABLE_PREPROCESSING': True,
    'PREPROCESSING_MODE': 'adaptive',
    'SAVE_PREPROCESSED': False,
    'PREPROCESSED_DIR': None,
}

# Make GROUND_TRUTH conditional based on INFERENCE_ONLY mode
if CONFIG['INFERENCE_ONLY']:
    CONFIG['GROUND_TRUTH'] = None

# ============================================================================
# PROMPT CONFIGURATION - Uses PROJECT_ROOT for subdirectory compatibility
# ============================================================================
PROMPT_CONFIG = {
    # Document type detection configuration
    'detection_file': str(PROJECT_ROOT / 'prompts/document_type_detection.yaml'),
    'detection_key': 'detection',
    
    # Extraction prompt file mapping (REQUIRED)
    'extraction_files': {
        'INVOICE': str(PROJECT_ROOT / 'prompts/generated/llama_invoice_prompt.yaml'),
        'RECEIPT': str(PROJECT_ROOT / 'prompts/generated/llama_receipt_prompt.yaml'),
        # BANK_STATEMENT: Only used if USE_SOPHISTICATED_BANK_EXTRACTION=False or as fallback
        'BANK_STATEMENT': str(PROJECT_ROOT / 'prompts/generated/llama_bank_statement_prompt.yaml')
    },
}

print("✅ Configuration set up successfully")
print(f"📂 Data directory: {CONFIG['DATA_DIR']}")
print(f"📊 Ground truth: {CONFIG['GROUND_TRUTH']}")
print(f"🤖 Model path: {CONFIG['MODEL_PATH']}")
print(f"📁 Output base: {CONFIG['OUTPUT_BASE']}")
print(f"🏠 Project root: {PROJECT_ROOT}")
print(f"🎯 Mode: {'Inference-only' if CONFIG['INFERENCE_ONLY'] else 'Evaluation mode'}")
print(f"🔧 Preprocessing: {'Enabled (' + CONFIG['PREPROCESSING_MODE'] + ')' if CONFIG['ENABLE_PREPROCESSING'] else 'Disabled'}")
print(f"🏦 V2 Bank Extraction: {'Enabled (multi-turn)' if CONFIG['USE_SOPHISTICATED_BANK_EXTRACTION'] else 'Disabled (single-turn)'}")
print(f"📐 Balance Correction: {'Enabled' if CONFIG['ENABLE_BALANCE_CORRECTION'] else 'Disabled'}")

## 3.5 Output Redirection Utility (Optional)

Use the `redirect_output_to_file()` context manager to save verbose output to a file instead of displaying in the notebook.

In [ ]:
#Cell 3.5
import sys
from contextlib import contextmanager
from pathlib import Path

@contextmanager
def redirect_output_to_file(filepath):
    """
    Context manager to redirect stdout to a file.
    
    Usage:
        log_path = OUTPUT_DIRS['base'] / f'batch_log_{BATCH_TIMESTAMP}.txt'
        with redirect_output_to_file(log_path):
            # Your verbose code here
            batch_results, processing_times, document_types_found = processor.process_batch(
                all_images, verbose=True
            )
        # Output is back to notebook
    
    Args:
        filepath: Path where output should be saved
        
    Yields:
        File handle for the log file
    """
    log_file = Path(filepath)
    log_file.parent.mkdir(parents=True, exist_ok=True)
    
    original_stdout = sys.stdout
    original_stderr = sys.stderr
    
    with open(log_file, 'w') as f:
        sys.stdout = f
        sys.stderr = f
        try:
            yield f
        finally:
            sys.stdout = original_stdout
            sys.stderr = original_stderr
    
    print(f"✅ Output saved to: {log_file}")

print("✅ Output redirection utility loaded")
print("📝 Use: with redirect_output_to_file('path/to/log.txt'): ...")

## 4. Output Directory Setup

In [ ]:
#Cell 4
# Setup output directories - Handle both absolute and relative paths

# Convert OUTPUT_BASE to Path and handle absolute/relative paths
OUTPUT_BASE = Path(CONFIG['OUTPUT_BASE'])
if not OUTPUT_BASE.is_absolute():
    # If relative, make it relative to current working directory
    OUTPUT_BASE = Path.cwd() / OUTPUT_BASE

BATCH_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

OUTPUT_DIRS = {
    'base': OUTPUT_BASE,
    'batch': OUTPUT_BASE / 'batch_results',
    'csv': OUTPUT_BASE / 'csv',
    'visualizations': OUTPUT_BASE / 'visualizations',
    'reports': OUTPUT_BASE / 'reports'
}

for dir_path in OUTPUT_DIRS.values():
    dir_path.mkdir(parents=True, exist_ok=True)

## 5. Model Loading

In [ ]:
#Cell 5
# Load model once for entire batch
rprint("[bold green]Loading model with robust multi-GPU detection...[/bold green]")

model, processor = load_llama_model_robust(
    model_path=CONFIG['MODEL_PATH'],
    use_quantization=CONFIG['USE_QUANTIZATION'],
    device_map=CONFIG['DEVICE_MAP'],
    max_new_tokens=CONFIG['MAX_NEW_TOKENS'],
    torch_dtype=CONFIG['TORCH_DTYPE'],
    low_cpu_mem_usage=CONFIG['LOW_CPU_MEM_USAGE'],
    verbose=CONFIG['VERBOSE']
)

# Model and processor will be used directly by DocumentAwareLlamaProcessor in batch processing
rprint("[bold green]✅ Model ready for document-aware processing with robust detection[/bold green]")

## 6. Image Discovery

In [ ]:
#Cell 6
# Discover and filter images - Handle both absolute and relative paths

# Convert DATA_DIR to Path and handle absolute/relative paths
data_dir = Path(CONFIG['DATA_DIR'])
if not data_dir.is_absolute():
    # If relative, make it relative to current working directory
    data_dir = Path.cwd() / data_dir

# Discover images from the resolved data directory
all_images = discover_images(str(data_dir))

# ============================================================================
# IMAGE PREPROCESSING (Optional)
# ============================================================================
# Apply preprocessing if enabled in CONFIG
if CONFIG['ENABLE_PREPROCESSING']:
    import tempfile
    from common.image_preprocessing import (
        enhance_statement_quality,
        enhance_for_llama,
        preprocess_statement_for_llama,
        adaptive_enhance,
        preprocess_recommended
    )
    
    # Map preprocessing mode to function
    preprocess_functions = {
        'light': enhance_statement_quality,
        'moderate': enhance_for_llama,
        'aggressive': preprocess_statement_for_llama,
        'adaptive': adaptive_enhance,
        'recommended': preprocess_recommended
    }
    
    preprocess_fn = preprocess_functions[CONFIG['PREPROCESSING_MODE']]
    preprocessed_images = []
    
    rprint(f"[cyan]🔧 Preprocessing {len(all_images)} images (mode: {CONFIG['PREPROCESSING_MODE']})[/cyan]")
    
    # Setup output directory for preprocessed images
    if CONFIG['SAVE_PREPROCESSED']:
        # Save to persistent directory
        preprocessed_dir = Path(CONFIG['PREPROCESSED_DIR'] or 'preprocessed_images')
        preprocessed_dir.mkdir(parents=True, exist_ok=True)
        rprint(f"[cyan]📁 Saving preprocessed images to: {preprocessed_dir}[/cyan]")
    else:
        # Save to temporary directory (preserves original filenames)
        preprocessed_dir = Path(tempfile.mkdtemp(prefix='preprocessed_'))
        rprint(f"[cyan]📁 Using temporary directory: {preprocessed_dir}[/cyan]")
    
    # Preprocess each image
    for img_path in all_images:
        original_filename = Path(img_path).name
        
        try:
            # Apply preprocessing function
            preprocessed_img = preprocess_fn(img_path)
            
            # Save with original filename (either to persistent or temp directory)
            preprocessed_path = preprocessed_dir / original_filename
            preprocessed_img.save(preprocessed_path)
            preprocessed_images.append(str(preprocessed_path))
            
        except Exception as e:
            rprint(f"[yellow]⚠️  Preprocessing failed for {original_filename}: {e}[/yellow]")
            rprint(f"[yellow]   Using original image instead[/yellow]")
            preprocessed_images.append(img_path)  # Fallback to original
    
    # Replace original image paths with preprocessed paths
    all_images = preprocessed_images
    rprint(f"[green]✅ Preprocessing complete ({len(preprocessed_images)} images)[/green]")
else:
    rprint("[dim]⏭️  Preprocessing disabled - using original images[/dim]")

# ============================================================================
# GROUND TRUTH LOADING
# ============================================================================
# Conditionally load ground truth only when not in inference-only mode
ground_truth = {}
if not CONFIG['INFERENCE_ONLY'] and CONFIG['GROUND_TRUTH']:
    # Convert GROUND_TRUTH to Path and handle absolute/relative paths
    ground_truth_path = Path(CONFIG['GROUND_TRUTH'])
    if not ground_truth_path.is_absolute():
        # If relative, make it relative to current working directory
        ground_truth_path = Path.cwd() / ground_truth_path
    
    # Load ground truth from the resolved path
    ground_truth = load_ground_truth(str(ground_truth_path), verbose=CONFIG['VERBOSE'])
    
    rprint(f"[green]✅ Ground truth loaded for {len(ground_truth)} images[/green]")
else:
    rprint("[cyan]📋 Running in inference-only mode (no ground truth required)[/cyan]")

# Apply filters (only if ground truth is available)
if CONFIG['DOCUMENT_TYPES'] and ground_truth:
    filtered = []
    for img in all_images:
        img_name = Path(img).name
        if img_name in ground_truth:
            doc_type = ground_truth[img_name].get('DOCUMENT_TYPE', '').lower()
            if any(dt.lower() in doc_type for dt in CONFIG['DOCUMENT_TYPES']):
                filtered.append(img)
    all_images = filtered

if CONFIG['MAX_IMAGES']:
    all_images = all_images[:CONFIG['MAX_IMAGES']]

rprint(f"[bold green]Ready to process {len(all_images)} images[/bold green]")
rprint(f"[cyan]Data directory: {data_dir}[/cyan]")
if not CONFIG['INFERENCE_ONLY'] and CONFIG['GROUND_TRUTH']:
    rprint(f"[cyan]Ground truth: {ground_truth_path}[/cyan]")
rprint(f"[cyan]Mode: {'Inference-only' if CONFIG['INFERENCE_ONLY'] else 'Evaluation mode'}[/cyan]")
for i, img in enumerate(all_images[:5], 1):
    print(f"  {i}. {Path(img).name}")
if len(all_images) > 5:
    print(f"  ... and {len(all_images) - 5} more")

In [ ]:
#Cell 7
# ============================================================================
# V2: BATCH PROCESSING WITH SOPHISTICATED BANK STATEMENT EXTRACTION
# ============================================================================
# This cell initializes the batch processor and sets up document routing:
#
# Document Flow:
# 1. Document type detection (Turn 0) - classify as INVOICE, RECEIPT, or BANK_STATEMENT
# 2. If BANK_STATEMENT:
#    - Turn 1: Header detection via BankStatementAdapter
#    - Turn 2: Adaptive extraction based on detected columns
# 3. If INVOICE/RECEIPT:
#    - Standard single-turn extraction
#
# This ensures NO duplicate extraction - each document is processed once.
# ============================================================================

# Initialize batch processor with simplified, clean architecture
batch_processor = BatchDocumentProcessor(
    model=model,
    processor=processor,
    prompt_config=PROMPT_CONFIG,
    ground_truth_csv=CONFIG['GROUND_TRUTH'],  # None in inference-only mode
    console=console,
    enable_math_enhancement=CONFIG['ENABLE_MATH_ENHANCEMENT']
)

# V2: Set up sophisticated bank statement extraction if enabled
if CONFIG.get('USE_SOPHISTICATED_BANK_EXTRACTION', False):
    rprint("[bold cyan]🏦 V2: Setting up sophisticated bank statement extraction...[/bold cyan]")
    
    # Create bank adapter for multi-turn extraction
    bank_adapter = BankStatementAdapter(
        model=model,
        processor=processor,
        verbose=CONFIG['VERBOSE'],
        use_balance_correction=CONFIG.get('ENABLE_BALANCE_CORRECTION', False),
    )
    
    # Store original processing method for non-bank documents
    original_process_llama = batch_processor._process_llama_image
    
    # Load detection config once for reuse
    from pathlib import Path as _Path
    import yaml as _yaml
    _detection_path = _Path(PROMPT_CONFIG["detection_file"])
    with _detection_path.open("r") as _f:
        _detection_config = _yaml.safe_load(_f)
    _detection_prompt_key = PROMPT_CONFIG.get("detection_key") or _detection_config.get("settings", {}).get("default_prompt", "detection")
    _doc_type_prompt = _detection_config["prompts"][_detection_prompt_key]["prompt"]
    _max_tokens = _detection_config.get("settings", {}).get("max_new_tokens", 50)
    
    def detect_document_type_only(image_path):
        """
        Detection-only function for Llama - returns just the document type.
        Does NOT perform extraction.
        """
        import torch
        from PIL import Image
        
        # Load image
        image = Image.open(image_path)
        
        # Create message structure
        messageDataStructure = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": _doc_type_prompt},
                ],
            }
        ]
        
        # Process input
        textInput = processor.apply_chat_template(messageDataStructure, add_generation_prompt=True)
        inputs = processor(image, textInput, return_tensors="pt")
        
        # Move to model device
        inputs = inputs.to(model.device)
        
        # Generate response
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=_max_tokens,
                do_sample=False,
                temperature=None,
                top_p=None,
            )
        response = processor.decode(output[0], skip_special_tokens=True)
        
        # Parse document type from response
        response_text = response.split("assistant")[-1].strip() if "assistant" in response else response
        response_upper = response_text.upper()
        
        # Debug: Show what the model returned
        print(f"  [DEBUG] Detection response: {response_text[:100]}...")
        
        # More specific matching - avoid false positives from "STATEMENT" in other contexts
        if "BANK STATEMENT" in response_upper or "BANK_STATEMENT" in response_upper:
            return "BANK_STATEMENT"
        elif "RECEIPT" in response_upper:
            return "RECEIPT"
        else:
            return "INVOICE"
    
    def enhanced_process_llama(image_path, verbose):
        """
        Enhanced processing with proper routing:
        - Turn 0: Document type detection (all documents)
        - If BANK_STATEMENT: BankStatementAdapter handles extraction (Turn 1 headers + Turn 2 extraction)
        - If INVOICE/RECEIPT: Standard extraction via original_process_llama
        
        NO duplicate extraction - each document processed exactly once.
        """
        from pathlib import Path
        
        # =====================================================================
        # TURN 0: Document Type Detection ONLY (no extraction yet)
        # =====================================================================
        if verbose:
            rprint(f"\n[cyan]📋 Turn 0: Document type detection for {Path(image_path).name}[/cyan]")
        
        try:
            doc_type = detect_document_type_only(image_path)
        except Exception as e:
            rprint(f"[red]Error in document type detection: {e}[/red]")
            raise
        
        if verbose:
            rprint(f"[green]✅ Detected: {doc_type}[/green]")
        
        # =====================================================================
        # ROUTING: Bank statements vs other documents
        # =====================================================================
        if doc_type.upper() == "BANK_STATEMENT":
            # -----------------------------------------------------------------
            # BANK STATEMENT: Use BankStatementAdapter (Turn 1 + Turn 2)
            # -----------------------------------------------------------------
            if verbose:
                rprint(f"[cyan]🏦 Routing to BankStatementAdapter for multi-turn extraction[/cyan]")
            
            try:
                # BankStatementAdapter handles:
                # - Turn 1: Header detection (identifies column names)
                # - Turn 2: Adaptive extraction based on detected columns
                schema_fields, metadata = bank_adapter.extract_bank_statement(image_path)
                
                # Build result structure compatible with BatchDocumentProcessor
                extraction_result = {
                    "extracted_data": schema_fields,
                    "raw_response": metadata.get("raw_responses", {}).get("turn1", ""),
                    "field_list": list(schema_fields.keys()),
                    "metadata": metadata,
                }
                
                # Create prompt name indicating strategy used
                strategy = metadata.get("strategy_used", "unknown")
                prompt_name = f"unified_bank_{strategy}"
                
                if verbose:
                    rprint(f"[green]  ✅ Strategy: {strategy}[/green]")
                    tx_count = len(schema_fields.get('TRANSACTION_DATES', '').split('|')) if schema_fields.get('TRANSACTION_DATES') != 'NOT_FOUND' else 0
                    rprint(f"[green]  ✅ Transactions extracted: {tx_count}[/green]")
                
                return doc_type, extraction_result, prompt_name
                
            except Exception as e:
                rprint(f"[yellow]⚠️  BankStatementAdapter failed: {e}[/yellow]")
                rprint(f"[yellow]   Falling back to standard extraction...[/yellow]")
                # Fall through to standard extraction
        
        # -----------------------------------------------------------------
        # INVOICE/RECEIPT (or bank fallback): Standard extraction
        # -----------------------------------------------------------------
        if verbose:
            rprint(f"[dim]📄 Using standard extraction for {doc_type}[/dim]")
        
        # Use original process (will re-detect but that's acceptable for non-bank)
        return original_process_llama(image_path, verbose)
    
    # Replace the processing method
    batch_processor._process_llama_image = enhanced_process_llama
    
    rprint("[green]✅ V2: Sophisticated bank statement extraction enabled[/green]")
    rprint("[cyan]   Flow: Detection → Route → Extract (no duplicate processing for bank)[/cyan]")
    rprint("[cyan]   Bank statements: Turn 0 (detect) → Turn 1 (headers) → Turn 2 (extract)[/cyan]")
    rprint("[cyan]   Invoice/Receipt: Turn 0 (detect) → Standard extraction[/cyan]")
    rprint(f"[cyan]   Balance correction: {'Enabled' if CONFIG.get('ENABLE_BALANCE_CORRECTION', False) else 'Disabled'}[/cyan]")
else:
    rprint("[dim]⏭️  V2: Sophisticated bank extraction disabled - using original single-turn[/dim]")

# Process batch with CONFIG verbose setting for detailed field comparison
batch_results, processing_times, document_types_found = batch_processor.process_batch(
    all_images, verbose=CONFIG['VERBOSE']
)

# Brief summary
rprint(f"[bold green]✅ Processed {len(batch_results)} images[/bold green]")
rprint(f"[cyan]Average time: {np.mean(processing_times):.2f}s[/cyan]")
if CONFIG['INFERENCE_ONLY']:
    rprint("[cyan]📋 Inference-only mode: No accuracy evaluation performed[/cyan]")
else:
    avg_accuracy = np.mean([r.get('evaluation', {}).get('overall_accuracy', 0) * 100 for r in batch_results if 'evaluation' in r])
    rprint(f"[cyan]Average accuracy: {avg_accuracy:.1f}%[/cyan]")
    
    # V2: Show bank statement specific results if sophisticated extraction was used
    if CONFIG.get('USE_SOPHISTICATED_BANK_EXTRACTION', False):
        bank_results = [r for r in batch_results if r.get('document_type', '').upper() == 'BANK_STATEMENT']
        if bank_results:
            bank_accuracy = np.mean([r.get('evaluation', {}).get('overall_accuracy', 0) * 100 for r in bank_results if 'evaluation' in r])
            rprint(f"[cyan]🏦 Bank statement accuracy (V2): {bank_accuracy:.1f}%[/cyan]")


## 8. Generate Analytics

In [ ]:
#Cell 8
# Create model-specific CSV file to match InternVL3 structure
# Define field columns (excluding validation-only fields for consistency with InternVL3)
# Use filtered field list (excludes validation-only fields: TRANSACTION_AMOUNTS_RECEIVED, ACCOUNT_BALANCE)
from common.config import get_v4_field_list, filter_evaluation_fields
FIELD_COLUMNS = filter_evaluation_fields(get_v4_field_list())

# Create comprehensive results data matching InternVL3 structure
llama_csv_data = []

for i, result in enumerate(batch_results):
    # Basic metadata
    image_name = Path(result['image_path']).name
    doc_type = result.get('document_type', '').lower()
    processing_time = processing_times[i] if i < len(processing_times) else 0
    
    # Extract fields from result
    extraction_result = result.get('extraction_result', {})
    extracted_fields = extraction_result.get('extracted_data', {})
    accuracy_data = result.get('evaluation', {})
    
    # Count fields
    total_fields = len(FIELD_COLUMNS)
    found_fields = sum(1 for field in FIELD_COLUMNS if extracted_fields.get(field, 'NOT_FOUND') != 'NOT_FOUND')
    field_coverage = (found_fields / total_fields * 100) if total_fields > 0 else 0
    
    # Handle both inference-only and evaluation modes
    if CONFIG['INFERENCE_ONLY'] or accuracy_data.get('inference_only', False):
        # Inference-only mode
        overall_accuracy = None
        fields_extracted = found_fields
        fields_matched = 0  # No matching in inference mode
        eval_total_fields = total_fields
    else:
        # Evaluation mode
        overall_accuracy = accuracy_data.get('overall_accuracy', 0) * 100 if accuracy_data else 0
        fields_extracted = accuracy_data.get('fields_extracted', 0) if accuracy_data else 0
        fields_matched = accuracy_data.get('fields_matched', 0) if accuracy_data else 0
        eval_total_fields = accuracy_data.get('total_fields', total_fields) if accuracy_data else total_fields
    
    # Create prompt identifier
    prompt_used = f"llama_{doc_type}" if doc_type else "llama_unknown"
    
    # Create row data
    row_data = {
        'image_file': image_name,
        'image_name': image_name,
        'document_type': doc_type,
        'processing_time': processing_time,
        'field_count': eval_total_fields,
        'found_fields': fields_extracted,
        'field_coverage': field_coverage,
        'prompt_used': prompt_used,
        'timestamp': datetime.now().isoformat(),
        'overall_accuracy': overall_accuracy,
        'fields_extracted': fields_extracted,
        'fields_matched': fields_matched,
        'total_fields': eval_total_fields,
        'inference_only': CONFIG['INFERENCE_ONLY']
    }
    
    # Add all field values
    for field in FIELD_COLUMNS:
        row_data[field] = extracted_fields.get(field, 'NOT_FOUND')
    
    llama_csv_data.append(row_data)

# Create DataFrame and save
llama_df = pd.DataFrame(llama_csv_data)
llama_csv_path = OUTPUT_DIRS['csv'] / f"llama_batch_results_{BATCH_TIMESTAMP}.csv"
llama_df.to_csv(llama_csv_path, index=False)

rprint("[bold green]✅ Llama model-specific CSV exported:[/bold green]")
rprint(f"[cyan]📄 File: {llama_csv_path}[/cyan]")
rprint(f"[cyan]📊 Structure: {len(llama_df)} rows × {len(llama_df.columns)} columns[/cyan]")
rprint("[cyan]🔗 Compatible with model_comparison.ipynb pattern: *llama*batch*results*.csv[/cyan]")

# Display sample of the exported data (conditional based on mode)
if CONFIG['INFERENCE_ONLY']:
    rprint("\n[bold blue]📋 Sample exported data (inference-only mode):[/bold blue]")
    sample_cols = ['image_file', 'document_type', 'processing_time', 'found_fields', 'field_coverage', 'inference_only']
    if len(llama_df) > 0:
        display(llama_df[sample_cols].head(3))
    else:
        rprint("[yellow]⚠️ No data to display[/yellow]")
else:
    rprint("\n[bold blue]📋 Sample exported data (first 3 rows, key columns):[/bold blue]")
    sample_cols = ['image_file', 'document_type', 'overall_accuracy', 'processing_time', 'found_fields', 'field_coverage']
    if len(llama_df) > 0:
        display(llama_df[sample_cols].head(3))
    else:
        rprint("[yellow]⚠️ No data to display[/yellow]")

    # Verification: Show accuracy values to confirm they're correct (evaluation mode only)
    rprint("\n[bold blue]🔍 Accuracy verification:[/bold blue]")
    for i, result in enumerate(batch_results[:3]):  # Show first 3
        evaluation = result.get('evaluation', {})
        original_accuracy = evaluation.get('overall_accuracy', 0)
        percentage_accuracy = original_accuracy * 100
        rprint(f"  {result['image_name']}: {original_accuracy:.4f} → {percentage_accuracy:.2f}%")

# Debug: Show extraction structure to verify extraction works
rprint("\n[bold blue]🔍 Data structure verification:[/bold blue]")
for i, result in enumerate(batch_results[:2]):  # Show first 2
    image_name = result['image_name']
    extraction_result = result.get('extraction_result', {})
    extracted_data = extraction_result.get('extracted_data', {})
    found_count = sum(1 for v in extracted_data.values() if v != 'NOT_FOUND')
    rprint(f"  {image_name}: Found {found_count} fields in extracted_data")
    if found_count > 0:
        # Show first few found fields
        found_fields = [(k, v) for k, v in extracted_data.items() if v != 'NOT_FOUND'][:3]
        for field, value in found_fields:
            rprint(f"    {field}: {value}")

# Create analytics
analytics = BatchAnalytics(batch_results, processing_times)

# Generate and save DataFrames
saved_files, df_results, df_summary, df_doctype_stats, df_field_stats = analytics.save_all_dataframes(
    OUTPUT_DIRS['csv'], BATCH_TIMESTAMP, verbose=CONFIG['VERBOSE']
)

# Display key results based on mode
rprint("\n[bold blue]📊 Results Summary[/bold blue]")
if CONFIG['INFERENCE_ONLY']:
    rprint("[cyan]📋 Running in inference-only mode - no accuracy metrics available[/cyan]")
    # Show extraction statistics instead
    rprint(f"[cyan]✅ Total images processed: {len(batch_results)}[/cyan]")
    rprint(f"[cyan]✅ Average fields found: {llama_df['found_fields'].mean():.1f}[/cyan]")
    rprint(f"[cyan]✅ Average field coverage: {llama_df['field_coverage'].mean():.1f}%[/cyan]")
else:
    display(df_summary)

## 9. Create Visualizations

In [ ]:
#Cell 9
# Create visualizations
visualizer = BatchVisualizer()

viz_files = visualizer.create_all_visualizations(
    df_results, 
    df_doctype_stats,
    OUTPUT_DIRS['visualizations'],
    BATCH_TIMESTAMP,
    show=False  # Disable display to reduce output
)

## 10. Generate Reports

In [ ]:
#Cell 10
# Generate reports
reporter = BatchReporter(
    batch_results, 
    processing_times,
    document_types_found,
    BATCH_TIMESTAMP
)

# Save all reports using CONFIG verbose setting
report_files = reporter.save_all_reports(
    OUTPUT_DIRS,
    df_results,
    df_summary,
    df_doctype_stats,
    CONFIG['MODEL_PATH'],
    {
        'data_dir': CONFIG['DATA_DIR'],
        'ground_truth': CONFIG['GROUND_TRUTH'],
        'max_images': CONFIG['MAX_IMAGES'],
        'document_types': CONFIG['DOCUMENT_TYPES']
    },
    {
        'use_quantization': CONFIG['USE_QUANTIZATION'],
        'device_map': CONFIG['DEVICE_MAP'],
        'max_new_tokens': CONFIG['MAX_NEW_TOKENS'],
        'torch_dtype': CONFIG['TORCH_DTYPE'],
        'low_cpu_mem_usage': CONFIG['LOW_CPU_MEM_USAGE']
    },
    verbose=CONFIG['VERBOSE']
)

## 11. Display Final Summary

In [ ]:
#Cell 11
# Display final summary
console.rule("[bold green]Batch Processing Complete[/bold green]")

total_images = len(batch_results)
successful = len([r for r in batch_results if 'error' not in r])
avg_accuracy = df_results['overall_accuracy'].mean() if len(df_results) > 0 else 0

rprint(f"[bold green]✅ Processed: {total_images} images[/bold green]")
rprint(f"[cyan]Success Rate: {(successful/total_images*100):.1f}%[/cyan]")
rprint(f"[cyan]Average Accuracy: {avg_accuracy:.2f}%[/cyan]")
rprint(f"[cyan]Output: {OUTPUT_BASE}[/cyan]")

# Display dashboard if available
dashboard_files = list(OUTPUT_DIRS['visualizations'].glob(f"dashboard_{BATCH_TIMESTAMP}.png"))
if dashboard_files:
    dashboard_path = dashboard_files[0]
    rprint("\n[bold blue]📊 Visual Dashboard:[/bold blue]")
    display(Image(str(dashboard_path)))
else:
    rprint(f"\n[yellow]⚠️ Dashboard not found in {OUTPUT_DIRS['visualizations']}[/yellow]")

In [ ]:
#Cell 12
# Calculate zero accuracy extractions
zero_accuracy_count = 0
zero_accuracy_images = []
total_evaluated = 0

for result in batch_results:
    # Check if evaluation data exists (not inference-only mode)
    evaluation = result.get("evaluation", {})

    if evaluation and not evaluation.get("inference_only", False):
        total_evaluated += 1
        accuracy = evaluation.get("overall_accuracy", 0)

        if accuracy == 0.0:
            zero_accuracy_count += 1
            zero_accuracy_images.append(
                {
                    "image_name": result.get("image_name", "unknown"),
                    "document_type": result.get("document_type", "unknown"),
                    "fields_extracted": evaluation.get("fields_extracted", 0),
                    "total_fields": evaluation.get("total_fields", 0),
                }
            )

# Display results
if total_evaluated > 0:
    console.rule("[bold red]Zero Accuracy Analysis[/bold red]")

    rprint(f"[cyan]Total documents evaluated: {total_evaluated}[/cyan]")
    rprint(f"[red]Documents with 0% accuracy: {zero_accuracy_count}[/red]")

    if zero_accuracy_count > 0:
        percentage = (zero_accuracy_count / total_evaluated) * 100
        rprint(f"[red]Zero accuracy rate: {percentage:.1f}%[/red]")

        rprint("\n[bold red]Documents with 0% Accuracy:[/bold red]")
        for i, img_info in enumerate(zero_accuracy_images, 1):
            rprint(f"  {i}. {img_info['image_name']} ({img_info['document_type']})")
            rprint(
                f"     Fields extracted: {img_info['fields_extracted']}/{img_info['total_fields']}"
            )
    else:
        rprint(
            "[green]✅ No documents with 0% accuracy - all extractions had some success![/green]"
        )
else:
    rprint(
        "[yellow]⚠️ Running in inference-only mode - no accuracy metrics available[/yellow]"
    )
